<figure>
  <IMG SRC="https://upload.wikimedia.org/wikipedia/commons/thumb/d/d5/Fachhochschule_Südwestfalen_20xx_logo.svg/320px-Fachhochschule_Südwestfalen_20xx_logo.svg.png" WIDTH=250 ALIGN="right">
</figure>

# Einführung Machine Learning
### Sommersemester 2022
Prof. Dr. Heiner Giefers

## Fashion MNIST mit Keras CNN

In diesem Notebook geht es um einen Datensatz, den wir schon aus vorherigen Aufgaben kennen, nämlich dem *Fashion MNIST* Datensatz.
An dieser Stelle wollen wir allerdings statt eine Multi-Klassen Logistischen Regression oder eines MLPs ein Faltungsnetz (oder auch *Convolutional Neural Network*, CNN) einsetzen.

Um den Code so kompakt wie möglich zu halten, verwenden wir die Keras API.
Zusätzlich benötigen wir Funktionen aus NumPy und Matplotlib.

In [ ]:
# TensorFlow und tf.keras
import tensorflow as tf
from tensorflow import keras

# Hilfsbibliotheken
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)
%load_ext tensorboard

Der Fashion MNIST Datensatz ist als Standard-Beispiel über die Keras API erhältlich.
Daher können wir ihn komfortabel über einen Keras-Aufruf herunterladen und direkt auf Trainings- und Testdatensätze aufteilen:

In [ ]:
fashion_mnist = keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

train_images = train_images.reshape(train_images.shape[0], train_images.shape[1], train_images.shape[2], 1)
test_images = test_images.reshape(test_images.shape[0], test_images.shape[1], test_images.shape[2], 1)

Nun machen wir aus dem kategorischen Integer Labeln einen Vektor aus *One-Hot* kodierten Labeln.

In [ ]:
train_labels_ohe = tf.one_hot(train_labels, depth=10)
test_labels_ohe = tf.one_hot(test_labels, depth=10)
print(test_labels_ohe[1])
print(test_labels[1])

Wie man sieht, haben wir 60.000 Bilder im Trainings- und 10.000 Bilder im Testdatensatz.

In [ ]:
print(train_images.shape)
print(test_images.shape)

Ein zufälliges Bild aus den Trainingsdaten sieht so aus:

In [ ]:
plt.imshow(train_images[np.random.randint(0,train_images.shape[0])].reshape(28,28), cmap='gray')

Die 28x28 Pixel großen Bilder bestehen aus 8-bit Grauwerten.
Um die Piwelwerte in den Bereich $[0,1]$ zu skalieren, teilen wir alle Pixel durch 255.

In [ ]:
#Pixelwerte nach [0,1] skalieren
train_images = train_images / 255.0
test_images = test_images / 255.0

Nun erzeugen wir ein sequentielles Keras Modell:

In [ ]:
model = keras.Sequential()

Zu diesem Modell können wir nun mit `model.add` Schichten hinzufügen.
Entwerfen Sie selbst eine Mehrschichtiges neuronales Netz.
Wählen Sie die Anzahl der Neuronen und die Aktivierungsfunktionen der einzelnen Schichten aus.

In [ ]:
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=2, padding='same', activation='relu', input_shape=(28,28,1))) 
model.add(tf.keras.layers.MaxPooling2D(pool_size=2))
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Conv2D(filters=16, kernel_size=2, padding='same', activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=2))
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(10, activation='softmax'))
model.summary()

In [ ]:
# Mit dieser Zelle koennen vorherige Log- und Modell-Dateien geloescht werden
#'''
!rm ./FashionMNIST_CNN.h5
!rm -rf ./logs/*
#'''

In [ ]:
import os
reuse = True
if(reuse == True and os.path.isfile("./FashionMNIST_CNN.h5")):
    model.load_weights("./FashionMNIST_CNN.h5")


import datetime
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)


Um das Modell zu erzeugen, müssen einige wesentliche Parameter definiert werden:

`optimizer` legt die Art Algorithmus zur Minimierung der Kostenfunktion fest, z.B.
- Adagrad
- Adam
- SGD
- RMSprop

`loss` bestimmt die Art der Kostenfunktion:
- `binary_crossentropy` für die Klassen 0 und 1
- `categorical_crossentropy` für one-hot-kodierte Klassen (Beispiel: $[0,0,1,0]$)
- `sparse_categorical_crossentropy` für integer-kodierte Klassen (Beispiel: $2$)





In [ ]:
#Modell erzeugen
model.compile(
    #optimizer='Adam',
    optimizer=tf.keras.optimizers.Adam(),
    #loss='categorical_crossentropy',
    loss= tf.keras.losses.CategoricalCrossentropy(),
    metrics=['accuracy'])

In [ ]:
#Modell trainieren
model.fit(train_images, train_labels_ohe,
          epochs=5,
          #validation_data=(test_images, test_labels),
          validation_split=0.2,
          callbacks=[tensorboard_callback]
         )

In [ ]:
model.save_weights("./FashionMNIST_CNN.h5")

Wir können uns nun den Verlauf des Trainings im **TensorBoard** ansehen.

In [ ]:
%tensorboard --logdir logs

**Aufgabe:** Evaulieren Sie das Modell mit dem Testdatensatz. Geben die *Classification Accuracy* aus.

In [ ]:
test_acc = None
# YOUR CODE HERE
raise NotImplementedError()
print('Test accuracy:', test_acc)

**Aufgabe:** Verwenden Sie nun statt der *One-Hot* kodierten Labels (z.B. `train_labels_ohe`) die originalen Labels mit den Klassen von $0$ bis $9$ (z.B. `train_labels`).
Was müssen See im Code umstellen, damit das Training für diese etwas anderen Daten funktioniert?

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

#!rm ./FashionMNIST_CNN_v2.h5
#!rm -rf ./logs/*


print('Test accuracy:', test_acc)


**Aufgabe:** Verbessern Sie die Qualität des CNNs. Sie können die Layer ändern, neue layer hinzufügen und/oder mehr Epochen trainieren.

### Referenzen
[1] [*Fashion-MNIST with tf.Keras*](https://blog.tensorflow.org/2018/04/fashion-mnist-with-tfkeras.html), TensorFlow Blog, 2018